# Agent to EDA assistent

In this second part, we will expand on the agent and add abilities (tools) to inspect a certain dataset.

This notebook will guide you through the steps, but some parts are left as exercises for you to complete.

> REMEMBER: The notebook is just there to experiment; the end result needs to be present in `chat_app.py` to be able to interactively chat.


# We need to add some necessary dependencies

uv add datasets pandas sqlalchemy

## Loading the dataset

The Titanic dataset is a wellknown standard dataset denoting the passengers of the Titanic. It is primarily used for causality regarding the survival rate.

To load the Titanic dataset, we use the `load_dataset` function from the `datasets` library. This function allows us to easily access and convert the dataset into a pandas DataFrame for further analysis. The Titanic dataset is available at [Hugging Face Datasets](https://huggingface.co/datasets/mstz/titanic).


In [1]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("mstz/titanic")["train"]
titanic_df: pd.DataFrame = dataset.to_pandas()

README.md:   0%|          | 0.00/707 [00:00<?, ?B/s]

d:\Repos\pkuppens\datacation-chatbot-workspace\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\piete\.cache\huggingface\hub\datasets--mstz--titanic. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


titanic.py:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/16.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/891 [00:00<?, ? examples/s]

## Using LangChain's SQL Database Integration

In this section, we will explore how to leverage LangChain's SQL Database integration to interact with and analyze structured data. LangChain provides a seamless way to connect to SQL databases, execute queries, and retrieve results for further processing.

The integration allows us to:

- Connect to various SQL databases using supported drivers.
- Perform complex queries to extract insights from the data.
- Combine SQL capabilities with LangChain's tools for advanced data manipulation and analysis.

For more details, refer to the [LangChain SQL Database Integration Documentation](https://python.langchain.com/docs/integrations/tools/sql_database/).


In [ ]:
from sqlalchemy import create_engine

# Create an in-memory SQLite database engine
engine = create_engine("sqlite:///:memory:")

# to_sql() parameters:
# - 'titanic': Name of the SQL table to create
# - engine: SQLAlchemy engine to use for database connection
# - if_exists='replace': If table exists, drop and recreate it (other options: 'fail', 'append')
# - index=False: Don't include the DataFrame's index as a column in the SQL table
#   We set index=False because:
#   1. The Titanic dataset already has meaningful columns
#   2. The auto-generated pandas index isn't meaningful for our analysis
#   3. Avoiding an extra column keeps the schema cleaner
# to_sql() returns None
# The data is stored in-memory only since we're using 'sqlite:///:memory:'
# To persist to disk, we would need to specify a file path like:
#   'sqlite:///titanic.db'

titanic_df.to_sql("titanic", engine, if_exists="replace", index=False)

## Make the tools available to the ReAct agent

- Ensure the agent can query the dataset using SQL commands.
- Test the agent's ability to summarize the dataset.
- Verify the agent can calculate statistics like mean, median, and mode.
- Check if the agent can handle missing data gracefully.

### Example questions to ask:

- "How many passengers survived the Titanic disaster?"
- "What is the average age of the passengers?"
- "What is the survival rate for male and female passengers?"
- "Show the top 5 oldest passengers and their survival status."
- Experiment with filtering data based specific conditions.


## Advanced

- Use a better prompt, check https://python.langchain.com/docs/integrations/tools/sql_database/#use-within-an-agent
- Implement https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html
- Use https://docs.chainlit.io/api-reference/elements/plotly to really show visual elements

  ```python
    show_bar_chart_column = None


    @tool
    def show_bar_chart(column: str):
        """Show a bar chart over the specified column."""
        global show_bar_chart_column

    ...

    @cl.on_message
    async def on_message(message: cl.Message):
        ...
        global show_bar_chart_column
        if show_bar_chart_column is not None:
            ...

  ```

- Split up functionality to use an MCP server (the chainlit app acts as a client) - https://github.com/langchain-ai/langchain-mcp-adapters
